In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sklearn
import re
import statistics 
import numpy as np
from IPython.display import clear_output, display
from datetime import datetime





In [2]:
df = pd.read_csv('Indian Earthquakes List Update_Magnitudes (1).csv',header=8, encoding = "ISO-8859-1", parse_dates=True)

/home/shivaansehgal/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,3,6,7,9,10,11,14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.columns = ( df.iloc[0].fillna('') + df.iloc[1].fillna(''))
df.columns = np.vectorize(lambda x: x.strip())(df.columns)

In [4]:
df = df.drop([0,1])
df = df.reset_index(drop=True)

In [5]:
df

,Sl. No.,YEAR,MONTH,DATE,ORIGIN TIME(UTC),(IST),MAGNITUDEMw,Mw,Mb,Mb,Ms,ML,LAT (N),LONG (E),DEPTH (km),INTENSITYMM,MMI,MME,LOCATION,REFERENCE
0,1,-2474,0,0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0,NaN,NaN,NaN,NaN,Dr STGR
1,2,-325,0,0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,71,24,0,NaN,NaN,NaN,NaN,Dr STGR
2,3,25,0,0,NaN,NaN,7.5,7.5,NaN,6.969202899,7.260619977,7.427072403,72.9,33.72,0,NaN,NaN,NaN,NaN,Dr STGR
3,4,26,5,10,08.19.10.0,NaN,6.1397,6.1397,5.737047101,5.737047101,5.698851894,5.999685205,17.3,80.1,NaN,NaN,NaN,NaN,NaN,NEIC
4,5,26,5,10,08.19.10.0,NaN,6.1397,6.1397,5.737047101,5.737047101,6.075520196,5.999685205,26,97,80,NaN,NaN,NaN,Myanmar,G-R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52984,52970,2019,7,28,19:24:08,00:54:08/2019-07-29/,3.2,3.2,NaN,3.07428,2.32377,2.91501,32.8°N,78.4°E,10,NaN,NaN,NaN,Jammu & Kashmir,NaN
52985,52971,2019,7,28,10:44:10,16:14:10,3.6,3.6,NaN,3.43659,2.78301,3.33473,25.5°N,90.4°E,70,NaN,NaN,NaN,"Distt. East Garo Hills ,Meghalaya",NaN
52986,52972,2019,7,28,21:25:06,7/29/2019 2:55,4,4,NaN,3.79891,3.24225,3.75446,23.2°N,86.5°E,22,NaN,NaN,NaN,"Distt. Puruliya, West Bengal",NaN
52987,52973,2019,7,29,3:33:53,9:03:53,4.3,4.3,NaN,4.07065,3.58668,4.06925,32.8°N,76.4°E,20,NaN,NaN,NaN,"Distt. Lahaul-Spiti,Himachal Pradesh",NaN


In [6]:
def ints(x):
    if pd.notna(x):
        if (type(x) == str and x.strip().isdigit()):
            return int(x.strip())
        elif (type(x) in [int, float]):
            return int(x)
        else:
            return -1
    return -1

In [7]:
def floats(x):
    if pd.notna(x):
        if (type(x) == str and x.strip().isdigit()):
            return int(x.strip())
        elif (type(x) in [int, float]):
            return float(x)
        else:
            return -1
    return -1

In [8]:
for col in ['YEAR','DATE','MONTH']:
#     df[col] = df[col].dropna()
    df[col] = df[col].apply(ints)

In [9]:
for col in ['Mw','MAGNITUDEMw']:
#     df[col] = df[col].dropna()
    df[col] = df[col].apply(floats)

In [10]:
df = df.drop(np.where((df['YEAR']<=0)|(df['MONTH']<=0) | (df['DATE']<=0))[0])

In [11]:
df = df.drop(df.index[df['Mw']<=0])

In [12]:
np.where(df['ORIGIN TIME(UTC)'].isna() & df['(IST)'].notna())

(array([], dtype=int64),)

In [13]:
np.where(df['ORIGIN TIME(UTC)'].notna() & df['(IST)'].isna())

(array([   15,    18,    22, ..., 19731, 19732, 19733]),)

In [14]:
np.where(df['MAGNITUDEMw'].notna() & df['Mw'].isna())

(array([], dtype=int64),)

In [15]:
np.where(df['MAGNITUDEMw'].isna() & df['Mw'].notna())

(array([], dtype=int64),)

In [16]:
def formatint(x):
    if pd.notna(x):
        if (type(x) == str and x.strip().isdigit()):
            return int(x.strip())
        elif (type(x) in [int, float]):
            return int(x)
        else:
#             print(x)
            return pd.NA
    return x

In [17]:
def formatfloat(x):
    if pd.notna(x):
        if (type(x) == str and x.strip().isdigit()):
            return int(x.strip())
        elif (type(x) in [int, float]):
            return float(x)
        else:
#             print(x)
            return pd.NA
    return x

In [18]:
df['MAGNITUDEMw'] = df['MAGNITUDEMw'].apply(formatfloat)

In [19]:
df

,Sl. No.,YEAR,MONTH,DATE,ORIGIN TIME(UTC),(IST),MAGNITUDEMw,Mw,Mb,Mb,Ms,ML,LAT (N),LONG (E),DEPTH (km),INTENSITYMM,MMI,MME,LOCATION,REFERENCE
332,318,1852,3,31,NaN,NaN,6.0,6.0,5.610507246,5.610507246,5.538461538,5.853095488,26.17,85.9,NaN,NaN,NaN,NaN,NaN,NEIC
449,435,1863,11,18,NaN,NaN,5.0,5.0,4.704710145,4.704710145,4.390355913,4.803777545,26.47,84.43,NaN,NaN,NaN,NaN,NaN,NEIC
572,558,1871,1,31,NaN,NaN,5.0,5.0,4.704710145,4.704710145,4.390355913,4.803777545,16.98,82.33,NaN,NaN,NaN,NaN,NaN,NEIC
590,576,1872,4,14,NaN,NaN,5.0,5.0,4.704710145,4.704710145,4.390355913,4.803777545,26.3,89.53,NaN,NaN,NaN,NaN,NaN,NEIC
645,631,1882,6,28,NaN,NaN,5.0,5.0,4.704710145,4.704710145,4.390355913,4.803777545,22.6,93.4,NaN,NaN,NaN,NaN,NaN,NEIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52984,52970,2019,7,28,19:24:08,00:54:08/2019-07-29/,3.2,3.2,NaN,3.07428,2.32377,2.91501,32.8°N,78.4°E,10,NaN,NaN,NaN,Jammu & Kashmir,NaN
52985,52971,2019,7,28,10:44:10,16:14:10,3.6,3.6,NaN,3.43659,2.78301,3.33473,25.5°N,90.4°E,70,NaN,NaN,NaN,"Distt. East Garo Hills ,Meghalaya",NaN
52986,52972,2019,7,28,21:25:06,7/29/2019 2:55,4.0,4.0,NaN,3.79891,3.24225,3.75446,23.2°N,86.5°E,22,NaN,NaN,NaN,"Distt. Puruliya, West Bengal",NaN
52987,52973,2019,7,29,3:33:53,9:03:53,4.3,4.3,NaN,4.07065,3.58668,4.06925,32.8°N,76.4°E,20,NaN,NaN,NaN,"Distt. Lahaul-Spiti,Himachal Pradesh",NaN


In [20]:
timestamp=[]
for index,row in df.iterrows():
    year=row['YEAR']
    month=row['MONTH']
    day=row['DATE']
    try:
        x = datetime(year,month, day)
    except:
        x = datetime(year,month, day-3)
    timestamp.append(datetime.timestamp(x))

df['TIMESTAMP']=timestamp

In [21]:
df = df[['YEAR', 'MONTH', 'DATE', 'ORIGIN TIME(UTC)','TIMESTAMP' ,'Mw', 'LAT (N)', 'LONG (E)',
       'DEPTH (km)', 'REFERENCE']]

In [22]:
df.head()

,YEAR,MONTH,DATE,ORIGIN TIME(UTC),TIMESTAMP,Mw,LAT (N),LONG (E),DEPTH (km),REFERENCE
332,1852,3,31,NaN,-3.715999e+09,6.0,26.17,85.9,NaN,NEIC
449,1863,11,18,NaN,-3.348885e+09,5.0,26.47,84.43,NaN,NEIC
572,1871,1,31,NaN,-3.121565e+09,5.0,16.98,82.33,NaN,NEIC
590,1872,4,14,NaN,-3.083635e+09,5.0,26.3,89.53,NaN,NEIC
645,1882,6,28,NaN,-2.761622e+09,5.0,22.6,93.4,NaN,NEIC


In [23]:
df['ORIGIN TIME(UTC)'].fillna('00:00:00',inplace=True)
df['DEPTH (km)'].fillna(0,inplace=True)
df['REFERENCE'].fillna('',inplace=True)

/home/shivaansehgal/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [24]:
df['ORIGIN TIME(UTC)']!=0

332      True
449      True
572      True
590      True
645      True
         ... 
52984    True
52985    True
52986    True
52987    True
52988    True
Name: ORIGIN TIME(UTC), Length: 20160, dtype: bool

In [25]:
def date2float(x):
    if type(x) in [int,float]:
        return x
    x = x.replace(' ',':')
    x = x.replace('.',':')
    flag = (x.find('PM') > -1)
    x = re.sub('[^0-9.:]*', '', x)
    
    x = np.array([ int(t) if t.isdigit() else -1 for t in x.split(':')])
    x = x[x>=0]
    
    x_ = np.zeros(4)
    try:
        x_[:x.shape[0]] = x
    except:
        print(x)
    
    if flag:
        x_[0] = x_[0] + (12 if x_[0]<12 else 0)
    
    return np.sum(x_ * np.array([3600,60,1,0.01]))

In [26]:
date2float('1:5:20 PM')

47120.0

In [27]:
df['ORIGIN TIME(UTC)'] = df['ORIGIN TIME(UTC)'].apply(date2float)
df['TIMESTAMP']+=df['ORIGIN TIME(UTC)']

/home/shivaansehgal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/shivaansehgal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
df

,YEAR,MONTH,DATE,ORIGIN TIME(UTC),TIMESTAMP,Mw,LAT (N),LONG (E),DEPTH (km),REFERENCE
332,1852,3,31,0.0,-3.715999e+09,6.0,26.17,85.9,0,NEIC
449,1863,11,18,0.0,-3.348885e+09,5.0,26.47,84.43,0,NEIC
572,1871,1,31,0.0,-3.121565e+09,5.0,16.98,82.33,0,NEIC
590,1872,4,14,0.0,-3.083635e+09,5.0,26.3,89.53,0,NEIC
645,1882,6,28,0.0,-2.761622e+09,5.0,22.6,93.4,0,NEIC
...,...,...,...,...,...,...,...,...,...,...
52984,2019,7,28,69848.0,1.564322e+09,3.2,32.8°N,78.4°E,10,
52985,2019,7,28,38650.0,1.564291e+09,3.6,25.5°N,90.4°E,70,
52986,2019,7,28,77106.0,1.564329e+09,4.0,23.2°N,86.5°E,22,
52987,2019,7,29,12833.0,1.564351e+09,4.3,32.8°N,76.4°E,20,


In [29]:
def NS(x):
    if type(x) in [int,float]:
        return x
    
    flag = x.find('S')>-1
    x = re.sub('[^0-9.]', '', x)
    return float(x) * (-1 if flag else 1)

In [30]:
def EW(x):
    if type(x) in [int,float]:
        return x
    
    flag = x.find('W')>-1
    x = re.sub('[^0-9.]', '', x)
    return float(x) * (-1 if flag else 1)

In [31]:
NS('28.5 South')

-28.5

In [32]:
df['LAT (N)'] = df['LAT (N)'].apply(NS)
df['LONG (E)'] = df['LONG (E)'].apply(EW)

/home/shivaansehgal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/shivaansehgal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
int('15 ')

15

In [34]:
df

,YEAR,MONTH,DATE,ORIGIN TIME(UTC),TIMESTAMP,Mw,LAT (N),LONG (E),DEPTH (km),REFERENCE
332,1852,3,31,0.0,-3.715999e+09,6.0,26.17,85.90,0,NEIC
449,1863,11,18,0.0,-3.348885e+09,5.0,26.47,84.43,0,NEIC
572,1871,1,31,0.0,-3.121565e+09,5.0,16.98,82.33,0,NEIC
590,1872,4,14,0.0,-3.083635e+09,5.0,26.30,89.53,0,NEIC
645,1882,6,28,0.0,-2.761622e+09,5.0,22.60,93.40,0,NEIC
...,...,...,...,...,...,...,...,...,...,...
52984,2019,7,28,69848.0,1.564322e+09,3.2,32.80,78.40,10,
52985,2019,7,28,38650.0,1.564291e+09,3.6,25.50,90.40,70,
52986,2019,7,28,77106.0,1.564329e+09,4.0,23.20,86.50,22,
52987,2019,7,29,12833.0,1.564351e+09,4.3,32.80,76.40,20,


In [35]:
# Add target column
# Add Last Earthquake at (X±0.5,Y±0.5)
# Add Number of Eathquake Before Last date
# Add Median Frequency at (X±0.5,Y±0.5)


latRange=0.5
longRange=0.5
lastEq=[]
medianFrequency=[]
numEq=[]
for index,row in df.iterrows():
    clear_output(wait=True)
    print(index,row)
    lat=row['LAT (N)']
    long=row['LONG (E)']
    timeList=df['TIMESTAMP'][df['LAT (N)']<lat+latRange][df['LAT (N)']>lat-latRange][df['LONG (E)']<long+longRange][df['LONG (E)']>long-longRange]    
    timeList=timeList.tolist()
    lastEq.append(max(timeList))
    numEq.append(len(timeList))
    timeList.sort()
    if len(timeList)>1:
        deqt=[] # Difference 
        for i in range(1,len(timeList)):
            deqt.append(timeList[i]-timeList[i-1])
        medianFrequency.append(statistics.median(deqt))
    else:
        medianFrequency.append(np.inf)

df['Last Eq'] = lastEq
df['Frequency'] = medianFrequency
df['numEq']=numEq
    



52988 YEAR                       2019
MONTH                         7
DATE                         31
ORIGIN TIME(UTC)          53402
TIMESTAMP           1.56456e+09
Mw                            3
LAT (N)                      20
LONG (E)                   72.8
DEPTH (km)                   10
REFERENCE                      
Name: 52988, dtype: object


/home/shivaansehgal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/shivaansehgal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/shivaansehgal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [36]:
df

,YEAR,MONTH,DATE,ORIGIN TIME(UTC),TIMESTAMP,Mw,LAT (N),LONG (E),DEPTH (km),REFERENCE,Last Eq,Frequency,numEq
332,1852,3,31,0.0,-3.715999e+09,6.0,26.17,85.90,0,NEIC,1.358079e+09,5.074078e+09,2
449,1863,11,18,0.0,-3.348885e+09,5.0,26.47,84.43,0,NEIC,-3.348885e+09,inf,1
572,1871,1,31,0.0,-3.121565e+09,5.0,16.98,82.33,0,NEIC,-3.121565e+09,inf,1
590,1872,4,14,0.0,-3.083635e+09,5.0,26.30,89.53,0,NEIC,5.113037e+10,2.946700e+07,15
645,1882,6,28,0.0,-2.761622e+09,5.0,22.60,93.40,0,NEIC,8.694549e+09,6.850364e+06,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52984,2019,7,28,69848.0,1.564322e+09,3.2,32.80,78.40,10,,1.635534e+09,3.405534e+07,9
52985,2019,7,28,38650.0,1.564291e+09,3.6,25.50,90.40,70,,1.602916e+09,8.817454e+06,27
52986,2019,7,28,77106.0,1.564329e+09,4.0,23.20,86.50,22,,1.564329e+09,4.036743e+07,5
52987,2019,7,29,12833.0,1.564351e+09,4.3,32.80,76.40,20,,5.670050e+10,3.719423e+06,63


In [37]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

NameError: name 'train_test_split' is not defined